In [4]:
'''import'''
import pandas as pd
import json
import os
import csv
import datetime
import calendar

### 메타데이터 읽기

In [5]:
# 전국 도시철도 역사정보 표준데이터
meta_file = 'data\metro-meta.csv'

# csv 파일을 pandas dataframe 형식으로 읽어들인다
meta_csv = pd.read_csv(meta_file)

# 서울 도시철도 데이터만 필터링한다 ('노선명' 필드에 '서울 도시철도'가 포함된 행만 선택)
meta_csv = meta_csv[meta_csv['노선명'].str.contains('서울 도시철도')]

# 역 번호와 역사명 출력해보기
#print(meta_csv.loc[:, ['역번호', '역사명']])

# 전체 행 갯수 출력
#print ("[*] 전체 행 갯수: ", len(meta_csv))

# 전체 행 출력
#for idx in meta_csv.index: print(meta_csv.loc[idx])
    
# 역 코드를 key로 하여 해당 역의 메타데이터 접근 (서울역의 역번호는 150이므로, 해당 값을 이용하여 row 선택)
test_obj = meta_csv[meta_csv['역번호']=='150']

station_num = test_obj['역번호'].values[0]
station_name = test_obj['역사명'].values[0]
station_name_en = test_obj['영문역사명'].values[0]
station_lat = test_obj['역위도'].values[0]
station_lon = test_obj['역경도'].values[0]

# 테스트 출력 (역번호, 역사명, 역사명-영문, 역위도, 역경도)
print(station_num, station_name, station_name_en, station_lat, station_lon)



150 서울역 Seoul Station 37.55575305 126.9724563


### 입력 데이터파일 전처리 (raw + meta)

In [13]:
DEBUG = False #True

years = ['2018', '2019', '2020']
#years = ['2020']
# 입력파일명 정의
raw_file_prefix = 'seoul-metro-logs-raw-'
# 결과파일명 정의
result_file_prefix = 'seoul-metro-logs2-'

for year in years:
    cur_file = os.path.join('data', raw_file_prefix + year + '.csv') 
    result_file = open(os.path.join('data', result_file_prefix+year+'.json'), 'w', encoding='utf-8')
    
    print(cur_file)
    
    f = open(cur_file, 'r')
    rdr = csv.reader(f)
    next(rdr) #Header 스킵
    
    num_line = 0
    
    dowEn = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    dowKo = ["월", "화", "수", "목", "금", "토", "일"]
    
    for line in rdr:  
        data = dict()
        
        # 현재 읽은 라인 (승차 데이터)
        data_in = line
        # 다음 라인 (하차 데이터)
        data_out = next(rdr)
        
        cur_date = list(map(lambda x: int(x) , data_in[0].split('-')))
        
        line_num = data_in[1].replace('호선', '')
        station_code = data_in[2]
        
        data['line_num'] = line_num
        data['location'] = dict()
        data['station'] = dict()
        data['day_of_week'] = dict()
        
        meta_obj = meta_csv[meta_csv['역번호']==station_code]
        
        # 예외 처리
        if meta_obj.empty: continue
        
        data['location']['lat'] = meta_obj['역위도'].values[0]
        data['location']['lon'] = meta_obj['역경도'].values[0]
        
        data['station']['name'] = meta_obj['역사명'].values[0]
        data['station']['en'] = meta_obj['영문역사명'].values[0]
        data['station']['code'] = station_code
        
        ## 시간별로 
        for h in range(19):
            # 날짜시간 조합
            
            
            _time = datetime.datetime(cur_date[0], cur_date[1], cur_date[2], h+5)
            cur_time = _time.astimezone().isoformat() # 새벽 5시부터 시작
            # 승하차 인원
            people_in = int(data_in[h+5])
            people_out = int(data_out[h+5])
        
            data['@timestamp'] = cur_time
            data['hour_of_day'] = h+5
            dow = _time.weekday()
            data['day_of_week']['en'] = dowEn[dow]
            data['day_of_week']['ko'] = dowKo[dow]
            data['day_of_week']['no'] = dow

            data['people'] = dict()
            data['people']['in'] = people_in
            data['people']['out'] = people_out
            data['people']['total'] = people_in + people_out
                        
            #print(data)
            # 파일로 출력
            result_file.write(json.dumps(data, ensure_ascii=False))
            result_file.write('\n')
            
        # 현재까지 읽어들인 라인 갯수
        num_line += 2
        if num_line % 10000 == 0: 
            print(num_line, ' processed.')
        
        if(DEBUG): 
            if (num_line == 100): break
            
    result_file.close()
        
    f.close()  

print ('[*] process done.')




data\seoul-metro-logs-raw-2018.csv
10000  processed.
20000  processed.
30000  processed.
40000  processed.
50000  processed.
60000  processed.
70000  processed.
80000  processed.
90000  processed.
100000  processed.
110000  processed.
120000  processed.
130000  processed.
140000  processed.
150000  processed.
160000  processed.
170000  processed.
180000  processed.
190000  processed.
200000  processed.
data\seoul-metro-logs-raw-2019.csv
10000  processed.
20000  processed.
30000  processed.
40000  processed.
50000  processed.
60000  processed.
70000  processed.
80000  processed.
90000  processed.
100000  processed.
110000  processed.
120000  processed.
130000  processed.
140000  processed.
150000  processed.
160000  processed.
170000  processed.
180000  processed.
190000  processed.
200000  processed.
data\seoul-metro-logs-raw-2020.csv
10000  processed.
20000  processed.
30000  processed.
40000  processed.
50000  processed.
60000  processed.
70000  processed.
80000  processed.
90000  pr